In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import os

# Loading the data

In [2]:
train_path = "/Users/andrei/code/images/images/train"

In [3]:
test_path = "/Users/andrei/code/images/images/validation"

In [4]:
y_train = []
X_train = []
for folder_path in os.listdir(train_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(train_path, folder_path)):
            img = load_img(os.path.join(os.path.join(train_path, folder_path), image_path), color_mode = "grayscale")
            X_train.append(img_to_array(img))
            y_train.append(os.path.basename(os.path.normpath(folder_path)))

In [10]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [34]:
y_test = []
X_test = []
for folder_path in os.listdir(test_path):
    if not folder_path.startswith("."):
        for image_path in os.listdir(os.path.join(test_path, folder_path)):
            img = load_img(os.path.join(os.path.join(test_path, folder_path), image_path), color_mode = "grayscale")
            X_test.append(img_to_array(img))
            y_test.append(os.path.basename(os.path.normpath(folder_path)))

In [35]:
X_test, y_test = np.array(X_test), np.array(y_test)

# Preprocessing

In [11]:
y_label_dict = {
    'angry':0,
    'disgust':0,
    'fear':0,
    'happy':2,
    'neutral':1,
    'sad':0,
    'surprise':0
}

In [31]:
y_df = pd.DataFrame(y_train)
y_df_num = y_df.applymap(lambda x : y_label_dict[x])
y_num = np.array(y_df_num)

y_train_cat = to_categorical(y_num)

In [36]:
y_df = pd.DataFrame(y_test)
y_df_num = y_df.applymap(lambda x : y_label_dict[x])
y_num = np.array(y_df_num)

y_test_cat = to_categorical(y_num)

# Baseline

In [23]:
baseline = np.max(pd.DataFrame(y_num).value_counts(normalize = True))

In [24]:
baseline

0.5785711807362687

# Model

In [25]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers, models

In [26]:
model2 = models.Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model2.add(Rescaling(1./255, input_shape=(48, 48, 1)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model2.add(layers.Conv2D(32, kernel_size=3, activation='relu'))
model2.add(layers.MaxPooling2D(3))

model2.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
model2.add(layers.MaxPooling2D(3))

model2.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
model2.add(layers.MaxPooling2D(1))

model2.add(layers.Flatten())
model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(3, activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2022-09-06 17:25:10.898692: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
from tensorflow.keras.callbacks import EarlyStopping

In [28]:
model2.fit(X_train, y_cat, batch_size = 32, epochs = 1000,
          callbacks=[EarlyStopping(patience = 10, restore_best_weights= True, monitor = "val_accuracy", mode = "max")],
         validation_split = 0.2, verbose = 1)

Epoch 1/1000
721/721 [==============================] - 15s 20ms/step - loss: 0.9084 - accuracy: 0.5867 - val_loss: 0.7885 - val_accuracy: 0.6618
Epoch 2/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.7432 - accuracy: 0.6827 - val_loss: 0.9289 - val_accuracy: 0.5736
Epoch 3/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.6919 - accuracy: 0.7026 - val_loss: 0.8350 - val_accuracy: 0.6286
Epoch 4/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.6609 - accuracy: 0.7209 - val_loss: 0.8217 - val_accuracy: 0.6101
Epoch 5/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.6407 - accuracy: 0.7307 - val_loss: 0.6718 - val_accuracy: 0.7216
Epoch 6/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.6175 - accuracy: 0.7423 - val_loss: 0.7310 - val_accuracy: 0.6822
Epoch 7/1000
721/721 [==============================] - 14s 20ms/step - loss: 0.5998 - accuracy: 0.7490 - val_loss: 0.9808 -

In [37]:
model2.evaluate(X_test, y_test_cat)

221/221 [==============================] - 1s 3ms/step - loss: 0.6729 - accuracy: 0.7164


[0.6728672981262207, 0.7163883447647095]